## Part 1: Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.layers import Dense

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [3]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head(5)

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [5]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Age', 'DistanceFromHome', 'Education', 'HourlyRate', 'EnvironmentSatisfaction', 'JobLevel', 'JobSatisfaction', 'JobInvolvement', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']

# Create X_df using your selected columns
X_df = attrition_df[X_columns]
X_df.head()

# Show the data types for X_df
X_df.dtypes

Age                         int64
DistanceFromHome            int64
Education                   int64
HourlyRate                  int64
EnvironmentSatisfaction     int64
JobLevel                    int64
JobSatisfaction             int64
JobInvolvement              int64
PercentSalaryHike           int64
PerformanceRating           int64
RelationshipSatisfaction    int64
YearsAtCompany              int64
YearsInCurrentRole          int64
YearsSinceLastPromotion     int64
YearsWithCurrManager        int64
dtype: object

In [6]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

# Create X_train and X_test using the selected columns
X_train = X_train[X_columns]
X_test = X_test[X_columns]

# Show shape of training and testing sets
print(X_train.shape)
print(X_test.shape)

(1102, 15)
(368, 15)


In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
# Nothing to convert

In [8]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
# Create a OneHotEncoder for the Department column
ohe_dept = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' to avoid dummy variable trap

# Fit the encoder to the training data
ohe_dept.fit_transform(attrition_df[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
dept_encoded_train = ohe_dept.transform(y_train[['Department']])
dept_encoded_test = ohe_dept.transform(y_test[['Department']])

# Convert encoded data to DataFrames
dept_encoded_train_df = pd.DataFrame(dept_encoded_train, columns=ohe_dept.get_feature_names_out(['Department']))
dept_encoded_test_df = pd.DataFrame(dept_encoded_test, columns=ohe_dept.get_feature_names_out(['Department']))

# Drop the original Department column and concatenate the new encoded columns
y_train = y_train.drop(columns=['Department']).reset_index(drop=True)
y_test = y_test.drop(columns=['Department']).reset_index(drop=True)
y_train = pd.concat([y_train, dept_encoded_train_df], axis=1)
y_test = pd.concat([y_test, dept_encoded_test_df], axis=1)

In [10]:
# Create a OneHotEncoder for the Attrition column
ohe_attr = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' to avoid dummy variable trap

# Fit the encoder to the training data
ohe_attr.fit_transform(attrition_df[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attr_encoded_train = ohe_attr.fit_transform(y_train[['Attrition']])
attr_encoded_test = ohe_attr.fit_transform(y_test[['Attrition']])

# Convert encoded data to DataFrames
attr_encoded_train_df = pd.DataFrame(attr_encoded_train, columns=ohe_attr.get_feature_names_out(['Attrition']))
attr_encoded_test_df = pd.DataFrame(attr_encoded_test, columns=ohe_attr.get_feature_names_out(['Attrition']))

# Drop the original Attrition column and concatenate the new encoded columns
y_train = y_train.drop(columns=['Attrition']).reset_index(drop=True)
y_test = y_test.drop(columns=['Attrition']).reset_index(drop=True)
y_train = pd.concat([y_train, attr_encoded_train_df], axis=1)
y_test = pd.concat([y_test, attr_encoded_test_df], axis=1)

In [11]:
unique, counts = np.unique(attr_encoded_test, return_counts=True)
print(dict(zip(unique, counts)))

{0.0: 300, 1.0: 68}


In [12]:
y_train.head()

,Department_Research & Development,Department_Sales,Attrition_Yes
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,0.0,1.0,1.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0


In [13]:
y_test.head()

,Department_Research & Development,Department_Sales,Attrition_Yes
0,1.0,0.0,1.0
1,0.0,1.0,1.0
2,1.0,0.0,1.0
3,0.0,1.0,0.0
4,1.0,0.0,0.0


## Create, Compile, and Train the Model

In [14]:
# Find the number of columns in the X training data
input_nodes = len(X_df.columns)

# Create the input layer
input_layer = tf.keras.layers.Input(shape=(15,))

# Create at least two shared layers
shared1 = Dense(32, activation='relu')(input_layer)
shared2 = Dense(16, activation='relu')(shared1)
shared3 = Dense(8, activation='relu')(shared2)

In [15]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
branch1 = Dense(8, activation='relu')(shared3)

# Create the output layer
output1 = Dense(1, name='Department', activation='softmax')(branch1)

In [16]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
branch2 = Dense(8, activation='relu')(branch1)

# Create the output layer
output2 = Dense(1, name='Attrition', activation='sigmoid')(branch2)

In [17]:
# Create the model
model = Model(inputs=input_layer, outputs=[output1, output2])

# Compile the model
model.compile(optimizer='adam',
              loss='mse',
              metrics={'Department': 'accuracy', 'Attrition': 'accuracy'})

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 15)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 32)                   512       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 16)                   528       ['dense[0][0]']               
                                                                                                  
 dense_2 (Dense)             (None, 8)                    136       ['dense_1[0][0]']             
                                                                                              

In [18]:
# Train the model
model.fit(X_train_scaled, [dept_encoded_train, attr_encoded_train], epochs=50, batch_size=8, validation_split=0.2)

Epoch 1/50
111/111 [==============================] - 0s 2ms/step - loss: 0.6836 - Department_loss: 0.5210 - Attrition_loss: 0.1626 - Department_accuracy: 0.4790 - Attrition_accuracy: 0.8513 - val_loss: 0.6710 - val_Department_loss: 0.5271 - val_Attrition_loss: 0.1439 - val_Department_accuracy: 0.4729 - val_Attrition_accuracy: 0.8281
Epoch 2/50
111/111 [==============================] - 0s 742us/step - loss: 0.6430 - Department_loss: 0.5210 - Attrition_loss: 0.1220 - Department_accuracy: 0.4790 - Attrition_accuracy: 0.8513 - val_loss: 0.6668 - val_Department_loss: 0.5271 - val_Attrition_loss: 0.1397 - val_Department_accuracy: 0.4729 - val_Attrition_accuracy: 0.8281
Epoch 3/50
111/111 [==============================] - 0s 696us/step - loss: 0.6379 - Department_loss: 0.5210 - Attrition_loss: 0.1169 - Department_accuracy: 0.4790 - Attrition_accuracy: 0.8513 - val_loss: 0.6643 - val_Department_loss: 0.5271 - val_Attrition_loss: 0.1372 - val_Department_accuracy: 0.4729 - val_Attrition_accur

111/111 [==============================] - 0s 763us/step - loss: 0.5763 - Department_loss: 0.5210 - Attrition_loss: 0.0553 - Department_accuracy: 0.4790 - Attrition_accuracy: 0.9387 - val_loss: 0.6956 - val_Department_loss: 0.5271 - val_Attrition_loss: 0.1685 - val_Department_accuracy: 0.4729 - val_Attrition_accuracy: 0.7919
Epoch 26/50
111/111 [==============================] - 0s 999us/step - loss: 0.5740 - Department_loss: 0.5210 - Attrition_loss: 0.0530 - Department_accuracy: 0.4790 - Attrition_accuracy: 0.9421 - val_loss: 0.6994 - val_Department_loss: 0.5271 - val_Attrition_loss: 0.1723 - val_Department_accuracy: 0.4729 - val_Attrition_accuracy: 0.7738
Epoch 27/50
111/111 [==============================] - 0s 872us/step - loss: 0.5713 - Department_loss: 0.5210 - Attrition_loss: 0.0503 - Department_accuracy: 0.4790 - Attrition_accuracy: 0.9432 - val_loss: 0.7050 - val_Department_loss: 0.5271 - val_Attrition_loss: 0.1778 - val_Department_accuracy: 0.4729 - val_Attrition_accuracy: 0.

111/111 [==============================] - 0s 707us/step - loss: 0.5517 - Department_loss: 0.5210 - Attrition_loss: 0.0307 - Department_accuracy: 0.4790 - Attrition_accuracy: 0.9682 - val_loss: 0.7148 - val_Department_loss: 0.5271 - val_Attrition_loss: 0.1876 - val_Department_accuracy: 0.4729 - val_Attrition_accuracy: 0.7919
Epoch 50/50
111/111 [==============================] - 0s 683us/step - loss: 0.5534 - Department_loss: 0.5210 - Attrition_loss: 0.0324 - Department_accuracy: 0.4790 - Attrition_accuracy: 0.9671 - val_loss: 0.7249 - val_Department_loss: 0.5271 - val_Attrition_loss: 0.1978 - val_Department_accuracy: 0.4729 - val_Attrition_accuracy: 0.7873


In [19]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'Department': dept_encoded_test, 'Attrition': attr_encoded_test})
test_results

12/12 [==============================] - 0s 716us/step - loss: 0.7187 - Department_loss: 0.5190 - Attrition_loss: 0.1997 - Department_accuracy: 0.4810 - Attrition_accuracy: 0.7772


[0.7186928391456604,
 0.51902174949646,
 0.19967111945152283,
 0.48097825050354004,
 0.7771739363670349]

In [20]:
# Print the accuracy for both department and attrition
print(f"Department: {test_results[3]}")
print(f"Attrition: {test_results[4]}")

Department: 0.48097825050354004
Attrition: 0.7771739363670349


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. No, the data is imbalanced. Accuracy works well when the dataset is balanced. However, with this data, Balanced Accuracy would be a better metric as it ensures each class is equally weighted and prevents dominance of the majority class.
2. I used sigmoid for attrition and softmax for department. Sigmoid is best for binary classification problems such as the attrition outcomes of yes or no. Softmax is best when outcomes are mutually exclusive and will output a probability distribution across the three departments in this data of R&D, Sales, or HR. This type of problem is a multi-class classification problem where softmax is better suited.
3. The model could be potentially improved with more data, balanced data, or feature engineering.